In [1]:
from pyspark.sql import SparkSession  
import numpy as np  
  
# Initialize Spark session with the configurations  
spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("SVD") \
    .config("spark.executor.memory", "5g") \
    .config("spark.driver.memory", "5g") \
    .config("spark.executor.cores", "4") \
    .config("spark.cores.max", "16") \
    .config("spark.executor.instances", "3") \
    .getOrCreate() 

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/15 10:54:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
#Final CODE:

In [4]:
from pyspark.sql import SparkSession
import numpy as np

def generatecorrelatedtestdata(spark, n_samples, n_features, target_r_squared=0.7):
    """
    Generates correlated test data and returns the true coefficients and an RDD.
    """
    # Generate random coefficients
    np.random.seed(31)
    true_coef = np.random.randn(n_features)
    X = np.random.randn(n_samples, n_features)
    X = np.column_stack((X[:,:-1], np.ones(X.shape[0]))) 
    y_clean = np.dot(X, true_coef)
    noise = np.random.randn(n_samples) * np.sqrt(1 - target_r_squared)
    y = y_clean + noise
    scaling_factor = np.sqrt(target_r_squared / (1 - target_r_squared))
    y = y_clean * scaling_factor + noise

    # Create an RDD of numpy arrays
    rdd = spark.sparkContext.parallelize(X)
    return  rdd,y

def svd(arr):
    """
    Function to calculate the SVD of a given nxm Matrix.
    """
    # Compute V and singular values
    eigval_V, V = np.linalg.eigh(arr.T @ arr)
    
    # Sort eigenvalues and eigenvectors in descending order
    idx = eigval_V.argsort()[::-1]
    eigval_V = eigval_V[idx]
    V = V[:, idx]
    
    # Compute singular values
    singval = np.sqrt(np.abs(eigval_V))
    
    # Compute U
    U = np.zeros((arr.shape[0], min(arr.shape[0], arr.shape[1])))
    for i in range(len(singval)):
        if singval[i] > 1e-15:  # Avoid division by zero
            U[:, i] = (arr @ V[:, i]) / singval[i]
    
    # Ensure V is returned as V.T
    Vt = V.T
    
    return U, singval, Vt

def parallelinvers(U, sigma, V):
    # Inverting sigma correctly
    sigma_inv = np.diag(1 / sigma)
    return V.T @ sigma_inv @ U.T

def betacalc(X,Y):
    """
    Calculates the beta values for the input matrix A (X + y).
    """
    
    # Step 1: Compute X^T @ X
    XtX = X.T @ X
    
    # Step 2: SVD decomposition of X^T @ X
    U, Sigma, Vt = svd(XtX)
    
    # Step 3: Calculate the inverse of XtX using SVD components
    parinv = parallelinvers(U, Sigma, Vt)
    
    # Step 4: Compute beta = (X^T X)^(-1) X^T Y
    beta = parinv @ X.T @ Y
    return beta


# Example usage (make sure a SparkSession 'spark' is already created)
rdd,y = generatecorrelatedtestdata(spark, 300, 100)
X = np.array(rdd.collect())  # Collect RDD into a numpy array
beta_values = betacalc(X,y)
XtX = X.T @ X
U, Sigma, Vt = svd(XtX)
print(beta_values)


[-6.55839490e-01 -4.91691210e-01  1.07914010e-01 -1.09752762e+00
 -3.30923572e-01 -1.17692928e+00 -1.13093615e+00  2.83152506e+00
 -1.10071927e+00 -1.22513413e-01  4.46032266e-01 -2.10802072e-01
 -1.52952995e+00 -1.41231571e+00  1.77440527e+00 -5.42673093e-01
 -2.69942196e+00  1.02783801e-01 -8.79170838e-01 -1.04398091e+00
 -1.42822311e+00  2.33527834e+00  3.03088843e+00 -6.46367791e-01
 -1.47830272e-01 -2.93175099e-01 -1.16380227e+00  7.72051879e-02
  1.02085647e+00 -3.51443067e-01  7.77464038e-01  1.06543317e+00
  1.73671000e+00 -1.43123725e-01 -6.62707861e-01 -2.87446994e-01
 -3.72306562e-02 -1.77799508e+00 -1.64898063e+00 -1.71799251e+00
  8.23904062e-03 -1.40547783e-01 -1.09257713e+00  2.38828375e+00
 -6.90495748e-01 -7.91397390e-01 -1.10719577e+00  1.95416017e+00
  2.15168012e+00 -9.21388111e-01  6.12199880e-01  9.68386421e-01
 -2.10319376e+00 -2.78159066e+00 -2.38329995e+00  3.68915648e+00
  3.93165953e-01  4.24978125e-01 -3.65674562e-01 -7.79804654e-01
  2.75540848e+00 -1.39467

In [7]:
from pyspark.sql import SparkSession
import numpy as np

def generatecorrelatedtestdata(spark, n_samples, n_features, target_r_squared=0.7):
    """
    Generates correlated test data and returns the true coefficients and an RDD.
    """
    np.random.seed(31)
    true_coef = np.random.randn(n_features)
    X = np.random.randn(n_samples, n_features)
    X = np.column_stack((X[:,:-1], np.ones(X.shape[0]))) 
    y_clean = np.dot(X, true_coef)
    noise = np.random.randn(n_samples) * np.sqrt(1 - target_r_squared)
    y = y_clean + noise
    scaling_factor = np.sqrt(target_r_squared / (1 - target_r_squared))
    y = y_clean * scaling_factor + noise
    rdd = spark.sparkContext.parallelize(X)
    return rdd, y

def distributed_svd(rdd):
    """
    Perform SVD using distributed computations.
    """
    # Compute X^T @ X in a distributed manner
    XtX = rdd.map(lambda row: np.outer(row, row)).reduce(lambda a, b: a + b)
    
    # Perform eigendecomposition
    eigval_V, V = np.linalg.eigh(XtX)
    
    # Sort eigenvalues and eigenvectors in descending order
    idx = eigval_V.argsort()[::-1]
    eigval_V = eigval_V[idx]
    V = V[:, idx]
    
    # Compute singular values
    singval = np.sqrt(np.abs(eigval_V))
    
    # We don't compute U here to save memory, as it's not needed for beta calculation
    
    return singval, V.T

def distributed_betacalc(rdd, y):
    """
    Calculate beta values using distributed computations without collecting large datasets.
    """
    # Broadcast y to all nodes
    y_broadcast = spark.sparkContext.broadcast(y)
    
    # Compute X^T @ X and X^T @ y in a distributed manner
    XtX_and_Xty = rdd.map(lambda row: (np.outer(row, row), row * y_broadcast.value)).reduce(
        lambda a, b: (a[0] + b[0], a[1] + b[1])
    )
    XtX, Xty = XtX_and_Xty
    
    # Perform SVD
    Sigma, Vt = distributed_svd(rdd)
    
    # Calculate beta using the pseudoinverse
    Sigma_inv = 1 / Sigma
    beta = Vt.T @ (Sigma_inv[:, np.newaxis] * (Vt @ Xty))
    
    return beta

# Main execution
spark = SparkSession.builder.appName("MemoryEfficientDistributedSVDBetaCalculation").getOrCreate()

# Generate correlated test data
rdd, y = generatecorrelatedtestdata(spark, 3000, 1000)

# Calculate beta values
beta_values = distributed_betacalc(rdd, y)

print("Beta values:")
print(beta_values)

# Clean up
spark.stop()

24/10/15 10:59:16 WARN TaskSetManager: Stage 6 contains a task of very large size (1473 KiB). The maximum recommended task size is 1000 KiB.
24/10/15 10:59:17 WARN TaskSetManager: Lost task 12.0 in stage 6.0 (TID 239) (172.18.0.3 executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/lib/python3.10/site-packages/pyspark/rdd.py", line 1919, in func
    the reduce function
  File "/opt/spark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_1822/4049677332.py", line 50, in <lambda>
ValueError: operands could n

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 6.0 failed 4 times, most recent failure: Lost task 1.3 in stage 6.0 (TID 275) (172.18.0.7 executor 2): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/lib/python3.10/site-packages/pyspark/rdd.py", line 1919, in func
    the reduce function
  File "/opt/spark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_1822/4049677332.py", 

In [33]:
from pyspark.sql import SparkSession
import numpy as np

def generatecorrelatedtestdata(spark, n_samples, n_features, target_r_squared=0.7):
    """
    Generates correlated test data and returns the true coefficients and an RDD.
    """
    # Generate random coefficients
    np.random.seed(31)
    true_coef = np.random.randn(n_features)
    X = np.random.randn(n_samples, n_features)
    X = np.column_stack((X[:, :-1], np.ones(X.shape[0])))
    y_clean = np.dot(X, true_coef)
    noise = np.random.randn(n_samples) * np.sqrt(1 - target_r_squared)
    y = y_clean + noise

    # Create an RDD of numpy arrays
    rdd = spark.sparkContext.parallelize(X)

    return  rdd,y

def svd(arr):
    """
    Function to calculate the SVD of a given nxn symmetric matrix.
    """
    # Compute the eigenvalue decomposition of the symmetric matrix 'arr'
    grammiam = arr.T @ arr
    
    eigvals, eigvecs = np.linalg.eigh(grammiam)
    
    # Arrange the singular values and eigenvectors in descending order
    idx = np.argsort(singvals)[::-1]
    eigvals = eigvals[idx]
    eigvecs = eigvecs[:, idx]
    
    # Since 'arr' is symmetric, U and V are the same
    U = eigvecs
    Vt = eigvecs.T
    
    return U, eigvals, Vt


def parallelinvers(U, sigma, Vt):  
    """Computes the inverse of a symmetric positive definite matrix using its SVD components."""  
      
    # Invert the singular values  
    sigma_inv = np.diag(1 / sigma)  
      
    # Compute the inverse: 
    A_inv = Vt.T @ sigma_inv @ U.T 
      
    return A_inv  


def betacalc(X,Y):
    """
    Calculates the beta values for the input matrix A (X + y).
    """ 

    Xtx = X.T @ X
    # Perform SVD on X^T X
    U, Sigma, Vt = svd(Xtx)
    
    # Compute the inverse of X^T X using SVD components
    XtX_inv = parallelinvers(U, Sigma,Vt)

    # Compute X^T Y (n_features,)

    XtX_invXT = XXt_inv @ X.T
    
    beta = XtX_invXT @ Y

    return beta



# Example usage (make sure a SparkSession 'spark' is already created)
rdd,y = generatecorrelatedtestdata(spark, 100, 10)
X = np.array(rdd.collect())  # Collect RDD into a numpy array
beta_values = betacalc(X,y)
print(beta_values)


NameError: name 'singvals' is not defined

In [29]:
def generatecorrelatedtestdata(spark, n_samples, n_features, target_r_squared=0.7):
    """
    Generates correlated test data and returns the true coefficients and an RDD.
    """
    # Generate random coefficients
    np.random.seed(31)
    true_coef = np.random.randn(n_features)
    X = np.random.randn(n_samples, n_features)
    X = np.column_stack((X[:, :-1], np.ones(X.shape[0])))
    y_clean = np.dot(X, true_coef)
    noise = np.random.randn(n_samples) * np.sqrt(1 - target_r_squared)
    y = y_clean + noise

    # Create an RDD of numpy arrays
    rdd = spark.sparkContext.parallelize(X)

    return  rdd,y

rdd, y = generatecorrelatedtestdata(spark,1000,100)

In [32]:
np.linalg.svd(rdd.collect())

SVDResult(U=array([[ 1.38846512e-02, -2.48175785e-03, -7.74676906e-03, ...,
         1.32726882e-02,  3.10237333e-03,  3.77001610e-02],
       [ 1.67121254e-02,  1.49412122e-02,  1.86982452e-02, ...,
        -3.83915648e-04, -4.09708396e-02, -4.12113843e-02],
       [ 1.17477436e-02,  4.56683652e-02,  8.57152409e-03, ...,
         2.40480534e-02,  2.18992889e-02, -1.41380133e-02],
       ...,
       [ 8.12997940e-02,  8.44867979e-03, -5.75382091e-02, ...,
         9.01717529e-01,  6.17502827e-03,  1.06530849e-02],
       [ 2.02385416e-02, -5.45070551e-02, -8.11903984e-03, ...,
         1.18839567e-02,  9.05068469e-01, -6.51533109e-03],
       [ 3.27344485e-02,  3.08695281e-03, -2.64591598e-02, ...,
         9.47778264e-03, -6.28284089e-03,  9.00687565e-01]]), S=array([41.24985246, 41.01446872, 40.51183091, 40.28519272, 39.77236025,
       39.58020959, 39.34382752, 38.81136015, 38.51930406, 38.34504931,
       38.14414641, 37.91298217, 37.8480604 , 37.44645155, 37.26803022,
       37.14